# !!! Not Working yet!!!

In [ ]:
# This is only to fix autocomplete in my version of Jupyter notebook.  Disregard if you don't have this problem
%config Completer.use_jedi = False

# This is to allow the use pyqtgraph for plots
%gui qt

# Building an example problem

In [ ]:
import qutip
from VV0_params import PL1,PL2
import numpy as np
import pyqtgraph as pg

def vv0_H(defect=PL2, B=[0,0,0]):
    #Construct the usual spin matrices
    Si = Sx,Sy,Sz = qutip.jmat(1)
    
    #Construct the Hamiltonian
    H0 = (2*np.pi)*defect['D']*Sz*Sz #Zero-Field splitting
    H = [H0]
    
    #Zeeman-Splitting
    for i, Bi in enumerate(B):
        if Bi:
            H.append([(2*np.pi)*defect['gamma_e']*Si[i], Bi])
    
    return H

psi0 = qutip.basis(3,1)
Bz = 0
w = (PL2['D']+Bz*PL2['gamma_e'])*2*np.pi
H = vv0_H(B=[lambda t, args: np.sin(w*t),0,lambda t, args: Bz])

In [ ]:
# Defining a plotting function
def plot(output):
    states = np.array([s.full() for s in output.states])[:,:,0]
    states = np.abs(states)**2

    labels = ['+1', ' 0', '-1']
    colors = ['b', 'g', 'r', 'c', 'y', 'w','m', 'b', 'g', 'r', 'c', 'm', 'y', 'w']
    plt = pg.plot()
    plt.addLegend()
    for i, ys in enumerate(states.T):
        plt.plot(ts, ys, pen=colors[i], name=labels[i])
    return plt

# Solving using Master Equation

In [ ]:
%%time
#For time independent hamiltonian
plus = (1/np.sqrt(2))*(qutip.basis(3,0) + qutip.basis(3,1))

# This is super quick!
ts = np.linspace(0,10e-6,1000)
H0 = [H[0]]
output = qutip.sesolve(H0, psi0, ts)


In [ ]:
%%time
# For 10us 600 points an lower is too little, so we do 700 points
ts = np.linspace(0,10e-6,700)

# This will take about 10sec to simulate
output = qutip.sesolve(H, psi0, ts)

In [ ]:
plot(output)

# Qutip Simulation using Floquet Formalism

Here we'll attempt to follow http://qutip.org/docs/4.1/guide/dynamics/dynamics-floquet.html to apply Floquet formalism to an isolated spin

In [ ]:
%%time
Si = Sx,Sy,Sz = qutip.jmat(1)
T = (2*np.pi)/w
Ts = np.linspace(0, T, 500 + 1)
ts = np.linspace(0,1e-6,10)

# Find the floquet modes and energies
f_modes_0, f_energies = qutip.floquet_modes(H,2*np.pi/w, args={})

# Precompute the mode table
f_modes_table_t = qutip.floquet_modes_table(f_modes_0, f_energies, ts , H, T, args={})

In [ ]:
%%time
# Here we can use as few points as we want
# Eg. start and finish times only
# ts = [0, 10e-6]
# Or we can also do the full time but with less point then before
ts = np.linspace(0,10e-6,100)

# Solve the floquet-markov master euqation
# This will only take ~250ms
output = qutip.fsesolve(H, psi0, ts, e_ops=[], T=T, args={})

In [ ]:
plot(output)

# Qutip Simulation from Waveform

In [ ]:
%gui qt
from wave_library import *
from wave import Channel
from wave_sim import Wave_Sim, Pulse_Type, Sim_Ch_Types
from VV0_params import PL1,PL2
import numpy as np
import pyqtgraph as pg

# First let's give names to our channels
chs = {
    'sg1_i'  : Channel.ch1_a,
    'sg1_g'  : Channel.ch1_m1,
    
    'sg1_q'  : Channel.ch2_a,
}
sg1_chs = [chs[x] for x in ['sg1_i','sg1_q', 'sg1_g']]

# Creating some basic pulses
mw = IQ_MW_Pulse(300e-9, *sg1_chs, freq_shift=0)
delay = Zero(t=1e-6, ch=chs['sg1_g'])

#Create the sequence
wave = delay + mw + delay + mw + delay

In [ ]:
f_plus = PL2['D']
Sim_B_chs = {
    chs['sg1_i']: Sim_Ch_Types.SG_I(f=f_plus),
    chs['sg1_q']: Sim_Ch_Types.SG_Q(f=f_plus),
}
sim = Wave_Sim(wave, B_chs=Sim_B_chs, B_static=[0,0,0])

In [ ]:
%%time
exp_rate = 1e9
sim_rate = 10e9
Bi_t = sim.generate_B_t(sim_rate=sim_rate*2, experiment_rate=exp_rate)
ts = wave.get_ts(rate=sim_rate)

for i, f in enumerate(Bi_t):
    Bi_t[i] = np.vectorize(f)

# ys = np.array(map(Bi_t[0], ts))
# ys = np.zeros(len(ts))
# for i,t in enumerate(ts):
#     ys[i] = Bi_t[0](t)
# ys
Bi_t[0](ts)

In [ ]:
pg.plot(ts, ys)

In [ ]:
class B_field(object):
    pass

class SG_IQ(B_field):
    def __init__(self, f, I, Q)